In [1]:
import scrapy
from scrapy.utils.response import open_in_browser
import requests
from scrapy.http import TextResponse,XmlResponse,HtmlResponse
import re

In [25]:
data=[
    {
        "id": "channel",
        "settings": [],
        "value": "season",
    },
    {
        "id": "searchTerm",
        "settings": [],
        "value": "Kabarettgipfel",
    },
    {
        "id": "shootingStatus",
        "settings": "OR",
        "value": [],
    },
    {
        "id": "movieType",
        "settings": "OR",
        "value": [],
    },
    {
        "id": "productionCountry",
        "settings": "OR",
        "value": [],
    },
    {
        "id": "region",
        "settings": "OR",
        "value": [],
    },
    {
        "id": "numberOfResults",
        "settings": [],
        "value": "",
    },
    {
        "id": "pageSize",
        "settings": [],
        "value": "50",
    },
    {
        "id": "sortOrder",
        "settings": [],
        "value": "score",
    },
    {
        "id": "pagination",
        "settings": [],
        "value": 1,
    },
    {
        "id": "displayStyle",
        "settings": [],
        "value": "card",
    }
]

In [26]:
data

[{'id': 'channel', 'settings': [], 'value': 'season'},
 {'id': 'searchTerm', 'settings': [], 'value': 'Kabarettgipfel'},
 {'id': 'shootingStatus', 'settings': 'OR', 'value': []},
 {'id': 'movieType', 'settings': 'OR', 'value': []},
 {'id': 'productionCountry', 'settings': 'OR', 'value': []},
 {'id': 'region', 'settings': 'OR', 'value': []},
 {'id': 'numberOfResults', 'settings': [], 'value': ''},
 {'id': 'pageSize', 'settings': [], 'value': '50'},
 {'id': 'sortOrder', 'settings': [], 'value': 'score'},
 {'id': 'pagination', 'settings': [], 'value': 1},
 {'id': 'displayStyle', 'settings': [], 'value': 'card'}]

In [27]:
res = requests.post('https://www.crew-united.com/en/SearchApi/search.asp',json=data)

In [28]:
res

<Response [200]>

In [29]:
xmlResponse = XmlResponse(res.url, body=res.text, encoding='utf-8')

In [7]:
response

NameError: name 'response' is not defined

In [8]:
open_in_browser(response)

NameError: name 'response' is not defined

In [30]:
html_string=xmlResponse.xpath("//searchResultRenderedContent/text()")[0].get()

In [31]:
response=HtmlResponse(res.url, body=html_string, encoding='utf-8')

In [32]:
len(response.xpath(".//li/div[@class='cu-ui-card-card opt-style-project js-cu-filter-result-element']"))

50

In [33]:
shows=response.xpath(".//li/div[@class='cu-ui-card-card opt-style-project js-cu-filter-result-element']//div[1]/span/a/parent::span/parent::div/parent::div")

In [34]:
len(shows)

47

In [35]:
show=shows[0]

In [36]:
show.xpath("./div/span/a")[0].xpath("./text()")

[<Selector xpath='./text()' data='Kabarettgipfel (2017)'>]

In [37]:
for show in shows:
    show_info={}
    heading=show.xpath("./div/span/a")[0].xpath("./text()").get()
    title=re.sub(r"\([0-9]{4}\)|\([0-9]{4}-[0-9]{4}\)",'',heading).strip()
    year=re.findall(r"\([0-9]{4}\)",heading)
    if year:
        year=year[0].replace("(","").replace(")","")
    else:
        year=""
    duration=re.findall(r"\([0-9]{4}-[0-9]{4}\)",heading)
    if duration:
        duration=duration[0].replace("(","").replace(")","").split("-")
    show_info['title']=title
    show_info['year']=year
    show_info['duration']=duration
    productions=show.xpath("./div//table[contains(@class,'opt-key-value')]//tr/th[text()='Production']/parent::tr/td/span/a/text()").extract()
    show_info['productions']=productions
    print(productions)

['e&a film GmbH']
['Viktor Nilsson Produktion [de]']
['TH Nürnberg Georg Simon Ohm [de]']
['Anderthalb Film- und Fernsehproduktion GmbH', 'IFBBW Institut für Bildbewegung GmbH & Co. KG']
['BR Bayerischer Rundfunk [de]']
['Mr. Bob Films GmbH [de]']
['Claus Stoermer Produktion [de]']
['Filmunique Medienproduktion GmbH']
['ServusTV Fernsehgesellschaft m.b.H [at]']
['MDR - Mitteldeutscher Rundfunk [de]']
['Kanu Film [de]']
['Caterina Woj Filmproduktion [de]']
['Nemada Film Produktion']
['schawa tv GmbH']
['The Metafiction Cabaret [de]']
['Bubbles Film GmbH']
['NAP - Filmworx KG [at]', 'Newactingproject']
['Kabuki Media [fr]']
['Ella Panther Filmworks [de]']
['Henrietta Langholz Produktion']
['The Metafiction Cabaret [de]']
['Fiction Films GmbH']
['Zero Film GmbH [de]']
['First Take Schauspielakademie [de]']
['Markus Kaatsch Produktion']
['ZDF - Zweites Deutsches Fernsehen [de]']
['Indeed! Agentur für Markeninszenierung']
['CFF   Christoph Felder Filmproduktion']
['Zero Film GmbH [de]']
['F

In [39]:
project={
    "title":"Kabarettgipfel",
    "produktion_jahr":"2011"
}
raw_shows = response.xpath(
                ".//li/div[@class='cu-ui-card-card opt-style-project js-cu-filter-result-element']//div[1]/span/a/parent::span/parent::div/parent::div")
shows = []
for show in raw_shows:
    show_info = {}
    heading = show.xpath("./div/span/a")[0].xpath("./text()").get()
    countries=show.xpath("./div/div//ul[@class='opt-pipe']/li[contains(text(),'Germany') or contains(text(),'DE')]/text()").extract()
    url=show.xpath("./div/span/a")[0].xpath("./@href").get()
    title = re.sub(r"\([0-9]{4}\)|\([0-9]{4}-[0-9]{4}\)", '', heading).strip()
    year = re.findall(r"\([0-9]{4}\)", heading)
    if year:
        year = year[0].replace("(", "").replace(")", "")
    else:
        year = ""
        duration = re.findall(r"\([0-9]{4}-[0-9]{4}\)", heading)
    if duration:
        duration = duration[0].replace("(", "").replace(")", "").split("-")
        show_info['title'] = title
        show_info['crew_united_url']=url
        show_info['year'] = year
        show_info['duration'] = duration
        productions = show.xpath(
                    "./div//table[contains(@class,'opt-key-value')]//tr/th[text()='Production']/parent::tr/td/span/a/text()").extract()
        show_info['productions'] = productions
        shows.append(show_info)
# print(shows)
relavent_shows = list(filter(lambda show: show['title'].lower() == project['title'].lower(), shows))
print(relavent_shows)
year_similar = list(filter(lambda show: show['year'].lower() == project['produktion_jahr'].lower(), relavent_shows))
found_project={}
if year_similar:
    found_project=year_similar[0]
else:
    if len(relavent_shows)==1:
        found_project=relavent_shows[0]
print(found_project)

[]
['Germany']
[]
['Germany']
['Germany']
[]
['Germany']
[]
[]
['Germany']
[]
[]
[]
['Germany']
['Germany']
['Germany']
[]
[]
['Germany']
['Germany']
['Germany']
['Germany']
[]
['Germany']
[]
[]
[]
['Germany']
[]
['Germany']
[]
['Germany']
[]
['Germany']
['Germany']
[]
[]
[]
['Germany']
['Germany']
['Germany']
['Germany']
['Germany']
['Germany']
[]
[]
[]
[{'title': 'Kabarettgipfel', 'crew_united_url': '/en/Kabarettgipfel__266598.html', 'year': '2017', 'duration': ['2018'], 'productions': ['e&a film GmbH']}]
{'title': 'Kabarettgipfel', 'crew_united_url': '/en/Kabarettgipfel__266598.html', 'year': '2017', 'duration': ['2018'], 'productions': ['e&a film GmbH']}


In [73]:
# if found_project:
#     requests.get('https://www.crew-united.com'+found_project['crew_united_url'],json=data)
res = requests.get('https://www.crew-united.com/en/Ellas-Baby__222017.html',json=data)

In [74]:
res

<Response [200]>

In [75]:
response=HtmlResponse(res.url, body=res.text, encoding='utf-8')

In [76]:
open_in_browser(response)

True

In [77]:
broadcast=response.xpath(".//div[@id='releases-tv']/table//tr/td[text()='First showing']/parent::tr/td[not(text()='First showing')]")

In [78]:
broadcast[0].xpath("./text()").get()

'Germany'

In [79]:
broadcast[1].xpath("./text()").get()

'ARD [de]'

In [80]:
broadcast[2].xpath("./text()").get()

'Friday, 13/10/2017'

In [81]:
productions=response.xpath(".//div[@id='production']/table/tbody/tr/td[2][not(contains(text(),'Koproduzent'))]/parent::tr/td[1]/span/a/text()").extract()

In [82]:
productions

['Südstern Film GmbH & Co. KG [de]', 'TV60Filmproduktion GmbH']

In [83]:
koproduzent_productions=response.xpath(".//div[@id='production']/table/tbody/tr/td[2][contains(text(),'Koproduzent')]/parent::tr/td[1]/span/a/text()").extract()

In [84]:
koproduzent_productions

[]